[Reference](https://jnikenoueba.medium.com/building-a-notification-service-with-fastapi-8cc51d2eb2a8)

In [1]:
pip install fastapi uvicorn requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.4 MB/s eta 0:00:00


In [2]:
pip install onesignal-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00


# 1. Set Up FastAPI

In [3]:
from fastapi import FastAPI, Depends
from notifications.routes import router as notifications_router
from auth.jwt_handler import JWTBearer

app = FastAPI()

app.include_router(notifications_router, prefix="/notifications", dependencies=[Depends(JWTBearer())])

@app.get("/")
def read_root():
    return {"message": "Welcome to the Notification Service!"}

ModuleNotFoundError: No module named 'notifications'

# 2. Models and Schemas

In [4]:
# notifications/models.py

from pydantic import BaseModel

class Notification(BaseModel):
    title: str
    message: str
    user_id: str

In [5]:
# notifications/schemas.py
from pydantic import BaseModel

class NotificationSchema(BaseModel):
    title: str
    message: str
    user_id: str

class NotificationResponse(BaseModel):
    success: bool
    message: str

# 3. Routes

In [8]:
# notifications/routes.py
from fastapi import APIRouter, HTTPException
from notifications.models import Notification
from notifications.schemas import NotificationSchema, NotificationResponse
from notifications.utils import send_notification

router = APIRouter()

@router.post("/", response_model=NotificationResponse)
async def create_notification(notification: NotificationSchema):
    success = send_notification(notification)
    if not success:
        raise HTTPException(status_code=500, detail="Notification failed")
    return NotificationResponse(success=True, message="Notification sent successfully")

# 4. Sending Notifications

In [9]:
# notifications/utils.py

import onesignal as onesignal_sdk
from notifications.models import Notification

ONESIGNAL_APP_ID = 'your_onesignal_app_id'
ONESIGNAL_API_KEY = 'your_onesignal_api_key'

onesignal_client = onesignal_sdk.Client(app_id=ONESIGNAL_APP_ID, rest_api_key=ONESIGNAL_API_KEY)

def send_notification(notification: Notification):
    notification_body = {
        'headings': {'en': notification.title},
        'contents': {'en': notification.message},
        'include_player_ids': [notification.user_id]
    }

    response = onesignal_client.send_notification(notification_body)
    return response.status_code == 200

# 5. Authentication

In [10]:
# auth/jwt_handler.py
import jwt
from fastapi import Request, HTTPException

SECRET_KEY = "your_secret_key"

def JWTBearer():
    def verify_jwt(request: Request):
        token = request.headers.get("Authorization")
        if not token:
            raise HTTPException(status_code=403, detail="Token is missing")
        try:
            payload = jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
        except jwt.ExpiredSignatureError:
            raise HTTPException(status_code=403, detail="Token has expired")
        except jwt.InvalidTokenError:
            raise HTTPException(status_code=403, detail="Invalid token")
    return verify_jwt

```
uvicorn main:app --reload
```